In [2]:
#!pip install google-cloud-translate
#!pip install --upgrade google-cloud-translate --force-reinstall
#reset session
#!pip install requests
#reset session
#!pip install gensim

In [3]:
from google.cloud import translate                                                                                 #lib to use google translation api
from nltk.tokenize import word_tokenize                                                                            #lib to use tokenize function of nltk
from nltk.corpus import stopwords                                                                                  #lib to use stop_words corpus of nltk
from nltk.stem import PorterStemmer                                                                                #lib to use Porter stemmer for stemming
import pandas as pd                                                                                                #lib to work with dataframes

In [4]:
data_raw = pd.read_csv('/content/datalab/data_csv/data.csv')                                                       #reading data.csv using pandas
#comments = data_raw["Comment"].tolist()                                                                           #converting comments in the csv file to list format for further computation
head = data_raw.head(573)
#head_list = head["Comment"].tolist()
#comments = data_raw["Comment"]

In [5]:
translated_comments = []          
def translate_text(text,target='en'): 
    """
    Function to translate the comments
    
    args: text: str
          target: str, default='en'
    """
    text = text.decode('utf-8')                                                                    
    translate_client = translate.Client()                                                                             #creating an object for the translation api
    result = translate_client.translate(text,target_language=target)                                                  #using the translation object
    translated_comments.append(result)                                                                                #storing results in translated_comments

In [6]:
_ = head["Comment"].apply(translate_text)                                                                             #applying the function to full columns of comments

In [7]:
temp = pd.DataFrame(translated_comments)                                                                              #converting the dictionary(from google translation api) obtained to dataFrame

In [8]:
def tokenize(string): 
  """
  Function to tokenize the comments
    
  args: text: str
  """
  tokenize_text = word_tokenize(string)
  return tokenize_text
temp["translatedText"] = temp["translatedText"].apply(tokenize)

In [9]:
def lower(strings):  
  """
  Function to set the strings of translatedText column in lower case
    
  args: text: str
  """
  stem_text = [string.lower() for string in strings]
  return stem_text
temp["translatedText"] = temp["translatedText"].apply(lower)

In [10]:
stop_words = set(stopwords.words('english'))

def stop_words_remove(strings):
  """
  Function to remove the regular english stop words
    
  args: text: str
  """
  stop = [string for string in strings if string not in stop_words]
  return stop
temp["translatedText"] = temp["translatedText"].apply(stop_words_remove) 

In [11]:
my_stop_words = [u'@',u',',u'.',u'quiqu',u'🙃',u'😍😢',u'😘😍👌',u'💜',u'i',u'🇨🇱',u'🔝🔝🔝',u'?',u'😍😍😍😍👏👏👏👏',u'😔',u'#',u'❤😍',u'🇨🇦',u'😍🌹😍🌹😍🌹😍🌹😍🌹😍🌹😍🌹😍🌹😍🌹😍🌹😍🌹😍🌹😍🌹😍🌹😍🌹😍🌹',u'😍😍😍❤️',u'😊',u'😭😭❤️❤️',u'🤣',u'❤️👏🏼',u'🤤',u'💜💜💜',u'🔊',u'❤️💘',u'\ufe0f',u'👏👏',u'🔥🔥🔥🔥🔥',u'😍😍😍🔝🔝❤️',u'❄️🇨🇦❄️',u'👇🏼🙅',u';',u'✔',u'!',u'&',u'i',u'39',u'*',u'much',u'',u'💚',u'☹️',u'❤️🖤',u'give',u'know',u'like',u'see',u'mona❤️❤️❤️',u'jo',u'awwww😍',u'🌹',u'hermosas😍',u'...',u':',u'(', u')',u'..',u'😍',u'😍😍',u'😍😍😍',u'♥',u'❤️',u'😘',u'💖',u'😂',u'love',u'💙',u'beauti',u'❤️🔥',u'quot',u'❤️❤️❤️❤️',u'quiqu',u'😭😭',u'❤️❤️❤️',u'😍😍😍😍',u'❤',u'💕',u'i',u'8',u'',u'would',u'😍❤️',u'😭',u'😂😂']

def my_stop_words_remove(strings):
  """
  Function to remove the customised stop words
    
  args: text: str
  """
  stop = [string for string in strings if string not in my_stop_words]
  return stop
temp["translatedText"] = temp["translatedText"].apply(my_stop_words_remove)

In [13]:
ps = PorterStemmer()                                                                                                  #object for the PorterStemmer
def stemmer(strings):  
  """
  Function to stem the tokenize strings of comments
    
  args: text: str
  """
  stem_text = [ps.stem(string) for string in strings]  
  return stem_text
temp["translatedText"] = temp["translatedText"].apply(stemmer)

In [14]:
processed_comments = temp.translatedText                                                                               #final processed comments

In [31]:
import gensim                                                                                                          #lib to apply topic modelling                                               
import gensim.corpora.dictionary                                                                                       #lib to map between words and their integer ids
dictionary = gensim.corpora.Dictionary(processed_comments)
dictionary.save('dictionary.dict')

In [33]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_comments]                                                   #converting dictionary to matrix
corpora.MmCorpus.serialize('corpus.mm', bow_corpus)

In [34]:
lda_model_bow = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary)                              #applying LDA using bag_of_words
#print(lda_model_bow)
lda_model_bow.save('topic.model')

In [35]:
#for i in range(0, lda_model_bow.num_topics):                                                                           #printing different topics in the document
#  print lda_model_bow.print_topic(i) 
#lda_model_bow.show_topics()

In [36]:
from gensim import corpora, models                                                                                      #lib to import modules to use tf,idf models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=5, id2word=dictionary)                            #applyting LDA using tf,idf

In [37]:
#for i in range(0, lda_model_tfidf.num_topics):                                                                          #printing different topics in the document
# print lda_model_tfidf.print_topic(i)
#lda_model_tfidf.show_topics()

In [41]:
#!pip install pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

In [43]:
d = gensim.corpora.Dictionary.load('dictionary.dict')
c = gensim.corpora.MmCorpus('corpus.mm')
lda = gensim.models.LdaModel.load('topic.model')
data=pyLDAvis.gensim.prepare(lda,c,d)

In [44]:
data

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      18.091949        1       1 -0.058281 -0.052068
7      12.169371        1       2 -0.091948  0.098634
2      11.910965        1       3 -0.002331  0.037590
5      10.753916        1       4  0.106142  0.081726
3       9.602929        1       5 -0.008603 -0.073040
1       9.294664        1       6  0.128955 -0.017249
9       8.035411        1       7 -0.048269  0.008555
6       7.408138        1       8 -0.011835 -0.037688
0       6.545835        1       9 -0.017115  0.041028
8       6.186820        1      10  0.003283 -0.087487, topic_info=     Category       Freq         Term      Total  loglift  logprob
term                                                              
262   Default  25.000000       beauti  25.000000  30.0000  30.0000
308   Default   8.000000        thank   8.000000  29.0000  29.0000
697   Default   6.000000         best   6.000000  28.0000  28.0000
432   Default   7.000000        thing   7.000000  27.0000  27.0000
52    Default   6.000000          die   6.000000  26.0000  26.0000
223   Default   7.000000         want   7.000000  25.0000  25.0000
339   Default   8.000000         kiss   8.000000  24.0000  24.0000
1173  Default   3.000000         poor   3.000000  23.0000  23.0000
1172  Default   6.000000         nice   6.000000  22.0000  22.0000
1193  Default  11.000000          dog  11.000000  21.0000  21.0000
552   Default   6.000000         take   6.000000  20.0000  20.0000
325   Default  10.000000         good  10.000000  19.0000  19.0000
277   Default   8.000000         life   8.000000  18.0000  18.0000
963   Default   5.000000        world   5.000000  17.0000  17.0000
380   Default   9.000000         girl   9.000000  16.0000  16.0000
1123  Default   8.000000     devermut   8.000000  15.0000  15.0000
624   Default   3.000000       pretti   3.000000  14.0000  14.0000
161   Default   6.000000         live   6.000000  13.0000  13.0000
795   Default   5.000000        great   5.000000  12.0000  12.0000
157   Default   4.000000        pussi   4.000000  11.0000  11.0000
189   Default   4.000000        women   4.000000  10.0000  10.0000
1071  Default  15.000000        quiqu  15.000000   9.0000   9.0000
716   Default   4.000000          men   4.000000   8.0000   8.0000
342   Default  11.000000          one  11.000000   7.0000   7.0000
532   Default   9.000000          say   9.000000   6.0000   6.0000
497   Default   6.000000       nobodi   6.000000   5.0000   5.0000
288   Default   3.000000         cute   3.000000   4.0000   4.0000
849   Default   4.000000         make   4.000000   3.0000   3.0000
1121  Default   4.000000         wish   4.000000   2.0000   2.0000
1012  Default   2.000000         leav   2.000000   1.0000   1.0000
...       ...        ...          ...        ...      ...      ...
743   Topic10   0.569401         impo   1.137308   2.0909  -5.4210
908   Topic10   0.569382         🕊️💸a   1.137274   2.0909  -5.4211
133   Topic10   0.569394         mala   1.137304   2.0909  -5.4210
578   Topic10   0.569403        charg   1.137323   2.0909  -5.4210
329   Topic10   0.569398        are-i   1.137316   2.0909  -5.4210
1176  Topic10   0.569387    rodríguez   1.137298   2.0909  -5.4210
802   Topic10   0.569392        child   1.137311   2.0909  -5.4210
185   Topic10   0.569374        glass   1.137276   2.0909  -5.4211
3     Topic10   0.569390       forget   1.137312   2.0909  -5.4210
764   Topic10   0.569379  alexcruz_95   1.137298   2.0909  -5.4211
1170  Topic10   0.569381         aunt   1.137305   2.0909  -5.4211
7     Topic10   0.569380    laaura_vl   1.137303   2.0909  -5.4211
1023  Topic10   1.087086       shadow   2.258452   2.0516  -4.7744
1012  Topic10   1.086881         leav   2.279048   2.0423  -4.7745
52    Topic10   2.122201          die   6.881812   1.6063  -4.1054
1173  Topic10   1.087082         poor   3.270247   1.6814  -4.7744
624   Topic10   1.087067  